Есть две идеи:
### 1-ая идея:

- Обучение:
Вытащить из корпуса все positive / negative слова
Векторизовать каждое слово с помощью word2vec
Написать простой классификатор (вектор - класс)

- Тестирование:
На вход поступает слово
Вектурезуем его с помощью word2vec и делаем predict


### 2-ая идея

- Обучение
Подать на word2vec сначала негативные слова, векторизовать, записать в массив. 
Потом подать на word2vec позитивные, векторизовать. 
Получится два облака.

- Тестирование
Далее входные слова сравнивать с каждым из этих двух облаков.
Берем входное слово, сравниваем его с каждым словом из позитивного облака, запиисываем расстояние. 
Потом сравниваем с каждым словом из негативного облака и тоже считаем расстояние. 
Сравниваем результаты по расстояниям по разным облакам.

In [159]:
# -*- coding: utf-8 -*-
from __future__ import division
import re
import random
import pandas as pd
import numpy as np
from lxml import etree

import pymorphy2
import math
from nltk.tokenize import TreebankWordTokenizer
from stop_words import get_stop_words

from text_cleanisation_NLP import text_to_words
import gensim

import matplotlib.pyplot as plt

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.base import TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.grid_search import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import fbeta_score, make_scorer
from sklearn import cross_validation

morph = pymorphy2.MorphAnalyzer()
tokenizer = TreebankWordTokenizer()

RUS_LETTERS = u'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'

In [164]:
def parse_xml(filename):
    with open(filename, encoding='utf-8') as f:
        xml = f.read()

    dict = {}
    text = []
    category = []
    sentiment = []
    term = []

    root = etree.fromstring(xml)
    for child in root:
        for aspect in child[3]:
            if aspect.attrib['type'] == 'implicit' and aspect.attrib['sentiment']!= 'both' and aspect.attrib['sentiment']!= 'neutral':
                text.append(child[2].text)
                category.append(aspect.attrib['category'])
                sentiment.append(aspect.attrib['sentiment'])
                term.append(aspect.attrib['term'])

    dict['text'] = text
    dict['category'] = category
    dict['sentiment'] = sentiment
    dict['term'] = term

    
    return dict

In [165]:
text_train = parse_xml('SentiRuEval_rest_markup_train.xml')
text_test = parse_xml('SentiRuEval_rest_markup_test.xml')

In [166]:
df1 = pd.DataFrame(text_train)
df2 = pd.DataFrame(text_test)

frames = [df1, df2]
df = pd.concat(frames)

In [167]:
df.head()

,category,sentiment,term,text
0,Food,positive,вкусное,"День 8-го марта прошёл, можно и итоги подвести..."
1,Interior,positive,уютный,"День 8-го марта прошёл, можно и итоги подвести..."
2,Food,positive,вкусные,Отмечали в этом ресторане день рождение на пер...
3,Interior,positive,уютно,Отмечали в этом ресторане день рождение на пер...
4,Interior,positive,красиво,Отмечали в этом ресторане день рождение на пер...


In [168]:
df.describe()

,category,sentiment,term,text
count,1367,1367,1367,1367
unique,5,2,505,371
top,Food,positive,вкусно,Отмечали свадьбу 15 марта. Ах-Ах!!! Замечатель...
freq,538,1235,137,14


In [169]:
df.groupby('sentiment').describe()

category    term  \
sentiment                           
negative  count       132     132   
          unique        5     105   
          top        Food  дорого   
          freq         54       5   
positive  count      1235    1235   
          unique        5     402   
          top        Food  вкусно   
          freq        484     137   

                                                               text  
sentiment                                                            
negative  count                                                 132  
          unique                                                 91  
          top     Посещали это заведение 2 раза, ощущения разные...  
          freq                                                    4  
positive  count                                                1235  
          unique                                                353  
          top     Отмечали свадьбу 15 марта. Ах-Ах!!! Замечатель...  
          freq                                                   14

In [170]:
print('Сделаем датасет сбалансированным')
df = pd.concat([df[df['sentiment'] == 'positive'].sample(frac=1)[:132], df[df['sentiment'] == 'negative']]).sample(frac=1)

Сделаем датасет сбалансированным


In [171]:
# Загрузим модель
m = 'ruscorpora_1_300_10.bin'
model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True)
model.init_sims(replace=True)

In [172]:
transit = {'ADJF':'ADJ', 'INFN':'VERB', 'ADVB':'ADV'}
robj = re.compile('|'.join(transit.keys()))

def cleanization(text):
    for line in text:
        # 1. Все буквы в нижний регистр
        text_text = text.lower()

        # 2. Удаление всех небукв
        letters_only = ''
        for _c in text_text:
            if _c in RUS_LETTERS:
                letters_only += _c
            else:
                letters_only += ' '

        # 3. Заменяем множественные пробелы
        while '  ' in letters_only:
            letters_only = letters_only.replace('  ', ' ')

        # 4. Токенизация
        word_list = tokenizer.tokenize(letters_only)

        # 5. Лемматизация
        clean_word_list = [morph.parse(word)[0].normal_form for word in word_list]  # лемматизация
    
        # 6. Удаление стоп-слов + добавление тегов - части речи
        meaningful_words = [str(word) + '_' + robj.sub(lambda m: transit[m.group(0)], str(morph.parse(word)[0].tag.POS)) for word in clean_word_list if word not in get_stop_words('ru')] # стоп-слова
        return ' '.join(meaningful_words)   # meaningful_words

In [173]:
from __future__ import division
def mean(a):
    return sum(a) / len(a)


# def numbers(text):
#     """Усредняет вектор слов."""
#     arr = []
#     clean_text = cleanization(text)
#     # для каждого слова в тексте выводим его вектор
#     for word in clean_text.split():
#     # есть ли слово в модели? Может быть, и нет
#         if word in model:
#             arr.append(model[word])
#     # print(len(list(map(mean, zip(*arr)))))
#     return list(map(mean, zip(*arr)))

def numbers(text):
    """Усредняет вектор слов."""
    arr = []
    clean_text = cleanization(text)
    # для каждого слова в тексте выводим его вектор
    for word in clean_text.split():
    # есть ли слово в модели? Может быть, и нет
        if word in model:
            arr.append(model[word])
    if len(list(map(mean, zip(*arr)))) != 0:
        return list(map(mean, zip(*arr)))
    else:
        return [0 for i in range(0, 300)]

# В функции надо указать, что делать с вектором, если не нашлось слова в моделе


class FunctionFeaturizer(TransformerMixin):
    """ Для создания своего вектора я использовала несколько фич: длину текста, количество заглавных букв
     (чем больше, тем обычно выше вероятность, что это спам), количество ! (в спам-сообщениях встречаются часто),
     количество чисел, сколько слов из словаря спам-слов (50 самых частых слов в коллекции спам-сообщений)"""
    def __init__(self, featurizers):
        self.featurizers = featurizers

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        fvs = []
        for datum in X:
            fv = numbers(datum)
            fvs.append(fv)
        return np.array(fvs)

In [174]:
X_train, X_test, y_train, y_test = train_test_split(df['term'], df['sentiment'], test_size=0.2)

In [175]:
X_train.head()

329         жесткое
494    пересоленная
664       улыбчивая
495     заветренные
288    по-домашнему
Name: term, dtype: object

In [176]:
y_train.head()

329    negative
494    negative
664    positive
495    negative
288    positive
Name: sentiment, dtype: object

In [177]:
X_train.describe()

count        211
unique       150
top       вкусно
freq          17
Name: term, dtype: object

In [178]:
def do_smth_with_model(data_train, class_train, data_test, class_test, steps):
    print('\nModel train')
    pipeline = Pipeline(steps=steps)

    cv_results = cross_val_score(pipeline,
                                 data_train,
                                 class_train,
                                 cv=10,
                                 scoring='accuracy',
                                )
    print(cv_results.mean(), cv_results.std())

    pipeline.fit(data_train, class_train)
    class_predicted = pipeline.predict(data_test)
    print(class_predicted)

    print(classification_report(class_test, class_predicted ))

    return pipeline, class_predicted

In [179]:
w2v_featurizer = FunctionFeaturizer(numbers)  # создание своего векторизатора

In [180]:
# Свой векторизатор
print('\nCustom Transformer')
pipeline, label_predicted = do_smth_with_model(X_train, y_train,
                                               X_test, y_test, 
                                               steps=[('custom', w2v_featurizer),
                                                      ('classifier', LogisticRegression())])


Custom Transformer

Model train
0.886363636364 0.0677682779414
['negative' 'negative' 'positive' 'negative' 'positive' 'positive'
 'positive' 'negative' 'positive' 'negative' 'negative' 'positive'
 'negative' 'positive' 'negative' 'positive' 'negative' 'negative'
 'positive' 'positive' 'positive' 'negative' 'negative' 'negative'
 'negative' 'negative' 'positive' 'positive' 'negative' 'positive'
 'negative' 'negative' 'negative' 'negative' 'negative' 'negative'
 'negative' 'negative' 'negative' 'positive' 'negative' 'negative'
 'negative' 'positive' 'positive' 'positive' 'positive' 'negative'
 'positive' 'positive' 'positive' 'positive' 'negative']
             precision    recall  f1-score   support

   negative       0.93      0.90      0.92        31
   positive       0.87      0.91      0.89        22

avg / total       0.91      0.91      0.91        53



In [182]:
# Свой векторизатор
print('\nCustom Transformer')
pipeline, label_predicted = do_smth_with_model(X_train, y_train,
                                               X_test, y_test, 
                                               steps=[('custom', w2v_featurizer),
                                                      ('classifier', ExtraTreesClassifier())])


Custom Transformer

Model train
0.816017316017 0.126021621154
['positive' 'negative' 'negative' 'negative' 'positive' 'positive'
 'positive' 'negative' 'positive' 'negative' 'negative' 'positive'
 'positive' 'positive' 'positive' 'positive' 'negative' 'negative'
 'positive' 'positive' 'positive' 'negative' 'positive' 'negative'
 'positive' 'negative' 'positive' 'positive' 'negative' 'positive'
 'negative' 'negative' 'negative' 'positive' 'negative' 'negative'
 'negative' 'negative' 'negative' 'positive' 'negative' 'negative'
 'negative' 'positive' 'positive' 'positive' 'positive' 'negative'
 'positive' 'positive' 'positive' 'positive' 'negative']
             precision    recall  f1-score   support

   negative       0.92      0.74      0.82        31
   positive       0.71      0.91      0.80        22

avg / total       0.83      0.81      0.81        53



In [183]:
# Сделать дата сет сбалансированным, предварительно смешав вместе с тест

In [184]:
# Свой векторизатор
print('\nCustom Transformer')
pipeline, label_predicted = do_smth_with_model(X_train, y_train,
                                               X_test, y_test, 
                                               steps=[('custom', w2v_featurizer),
                                                      ('classifier', LogisticRegression())])


Custom Transformer

Model train
0.886363636364 0.0677682779414
['negative' 'negative' 'positive' 'negative' 'positive' 'positive'
 'positive' 'negative' 'positive' 'negative' 'negative' 'positive'
 'negative' 'positive' 'negative' 'positive' 'negative' 'negative'
 'positive' 'positive' 'positive' 'negative' 'negative' 'negative'
 'negative' 'negative' 'positive' 'positive' 'negative' 'positive'
 'negative' 'negative' 'negative' 'negative' 'negative' 'negative'
 'negative' 'negative' 'negative' 'positive' 'negative' 'negative'
 'negative' 'positive' 'positive' 'positive' 'positive' 'negative'
 'positive' 'positive' 'positive' 'positive' 'negative']
             precision    recall  f1-score   support

   negative       0.93      0.90      0.92        31
   positive       0.87      0.91      0.89        22

avg / total       0.91      0.91      0.91        53

